# KNN Model & Model Comparison
This notebook explores KNN as a distance-based learning approach and compares it with previous models.


In [4]:
import numpy as np
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix


In [5]:

data_path="C:/Users/prasad/OneDrive/Documents/student-performance-risk-prediction/data/raw/student-mat.csv"
df=pd.read_csv(data_path,sep=';')

# Create target variable
def assign_risk(grade):
    if grade < 10:
        return "High"
    elif grade < 15:
        return "Medium"
    else:
        return "Low"

df["risk_level"] = df["G3"].apply(assign_risk)

# Remove leakage features
df = df.drop(columns=["G1", "G2", "G3"])


In [6]:
X = df.drop(columns=["risk_level"])
y=df["risk_level"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [11]:
numeric_features = X.select_dtypes(exclude="object").columns
categorical_features = X.select_dtypes(include=["object"]).columns

numeric_features, categorical_features


(Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
        'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences'],
       dtype='object'),
 Index(['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob',
        'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities',
        'nursery', 'higher', 'internet', 'romantic'],
       dtype='object'))

## Why Scaling is Mandatory for KNN

KNN uses distance calculations.
If features are on different scales, features with larger ranges dominate distance calculations.

Therefore:
- Numeric features are scaled
- Categorical features are encoded


In [14]:
knn_preprocessor=ColumnTransformer(
    transformers=[
        ("cat",OneHotEncoder(drop="first",handle_unknown="ignore"),categorical_features),
        ("num",StandardScaler(),numeric_features)
    ])

## KNN Model Intuition

KNN classifies a student based on the majority class among its nearest neighbors.
The value of K controls the balance between noise sensitivity and generalization.


In [19]:
knn_model=Pipeline(steps=[
    ("preprocessing",knn_preprocessor),
    ("classifier",KNeighborsClassifier(n_neighbors=7))
     ])

In [20]:
knn_model.fit(X_train,y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessing', ...), ('classifier', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('cat', ...), ('num', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transform

In [22]:
y_pred_knn=knn_model.predict(X_test)
print(classification_report(y_test, y_pred_knn))
print(confusion_matrix(y_test, y_pred_knn))

              precision    recall  f1-score   support

        High       0.29      0.19      0.23        26
         Low       0.25      0.13      0.17        15
      Medium       0.48      0.68      0.57        38

    accuracy                           0.42        79
   macro avg       0.34      0.34      0.32        79
weighted avg       0.38      0.42      0.38        79

[[ 5  2 19]
 [ 4  2  9]
 [ 8  4 26]]


## KNN Results Interpretation

The KNN model struggles to achieve strong performance, particularly for minority risk classes.
This is expected due to:
- Class imbalance
- High-dimensional encoded data
- Sensitivity to noisy neighbors

These results highlight the limitations of KNN for this dataset.


## Model Comparison

All evaluated models are compared based on performance, stability, and interpretability.
we select Controlled Decision Tree 
